In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size=8
batch_size=4
max_iters=1000
learning_rate=3e-4

cuda


In [10]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars=sorted(set(text))
print(chars)
vocab_size=len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [11]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data=torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [12]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

tensor([ 42382, 126496, 118700,  73228])
inputs:
tensor([[58,  1, 62, 73,  1, 57, 62, 72],
        [66,  1, 65, 68, 68, 64,  1, 54],
        [36, 58, 73,  1, 74, 72,  1, 54],
        [67, 60,  1, 54, 67, 57,  0, 73]], device='cuda:0')
targets:
tensor([[ 1, 62, 73,  1, 57, 62, 72, 54],
        [ 1, 65, 68, 68, 64,  1, 54, 67],
        [58, 73,  1, 74, 72,  1, 54, 65],
        [60,  1, 54, 67, 57,  0, 73, 76]], device='cuda:0')


In [31]:
class BigramModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits=self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits=logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index=torch.cat((index,index_next),dim=1)
        return index
    
model = BigramModel(vocab_size)
m=model.to(device)

context=torch.zeros((1,1),dtype=torch.long, device=device)
generated_chars=decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


jR)_Z&.?,D5﻿.jQcYMK[FdjUMqhni 96UuBQ,Hl;jKiDBZBQrf-DzJe9xWRG19SneLJ(tKxQO
PhqSouRjR*?g],4 qs_N Vy﻿b;wqdUmLX(JI[t0W(NgvDdeSefD:QS7Xl(QpHvCHxWUB!A7sG!F1ABI
_lWPefZx]*OFL2-2n-o;h&3cp"a:L(pgpmwVW
CtSWgk(J2-﻿iqhEci?KR-!'S "YjSY&)JLKN)xvdilqToaDf.VH)V;KF743G.*G1MxOGxOmtp9IS'UKV.
:1M0,QFz"wcTx
xaD3q2VEssM;VB!hjm_HvTH0[x;P:Lc(0QHH&5VBN﻿nfOl cUQHW
!yp'﻿7E0DB!MlEImFC'e)xW(5I!(VilefYi8G9!7["7;Ohj0H]xkMh;I
Q9tKRd72)yZxx;KRSR E8h6tEV,2-,Ham ;EaA03,XUAMV"zwSNxW*N8MR-F ylT3ouXPalIH?r6VrT
AQO
h
N-qwRRuE
1-n91N4


In [33]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # if iter % eval_iters == 0:
    #     losses = estimate_loss()
    #     print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([ 42701, 108279,  30252,   4944])
tensor([ 81519, 177310,  28891, 134382])
tensor([104728,  20040, 171240, 104435])
tensor([153338, 114970, 159141, 133479])
tensor([ 68979, 181287,  95331, 169638])
tensor([83158, 39666, 31734, 60150])
tensor([140739, 151678, 102623,   7404])
tensor([ 22255,   1066, 152844, 160039])
tensor([151551,  16339,  79382,  62750])
tensor([183129,  27393, 144192,  97998])
tensor([104370, 139383, 149335, 132743])
tensor([120497, 158029, 130533,  15565])
tensor([ 49170,  39183,  32681, 112749])
tensor([131295, 175250, 133666,  73583])
tensor([154604, 100226,  56636,  22257])
tensor([183531,  56163,  56896,  45787])
tensor([ 83967,  22539,  55231, 105301])
tensor([ 15391,   4454, 152219, 143030])
tensor([ 55457, 129586,  71561,  33844])
tensor([180283, 111038, 163876, 105143])
tensor([ 41148, 126030,  66216, 123354])
tensor([146886,  35700, 184673, 143197])
tensor([ 10057, 138693,  28851,  10471])
tensor([  3188, 103859,  10210, 170555])
tensor([117255, 1306